Working example as of 2023/03/29 to connect transformers using accelerate to TPUs in colab with its limited RAM.

REF:
https://huggingface.co/docs/accelerate/v0.18.0/en/usage_guides/big_modeling#handling-big-models-for-inference
https://huggingface.co/docs/accelerate/concept_guides/training_tpu 

In [ ]:
!pip install transformers accelerate cloud-tpu-client==0.10 torch==2.0.0 torchvision==0.15.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp39-cp39-linux_x86_64.whl

In [ ]:
!git clone https://huggingface.co/sgugger/sharded-gpt-j-6B

In [ ]:
import os
import torch_xla.core.xla_model as xm
device = xm.xla_device()
print(device)

In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from accelerate import Accelerator, init_empty_weights, load_checkpoint_and_dispatch
import torch

model_name_or_path = 'EleutherAI/gpt-j-6B'
tokenizer_name = model_name_or_path

config = AutoConfig.from_pretrained(model_name_or_path)

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

with init_empty_weights():
   model = AutoModelForCausalLM.from_config(config)
model.tie_weights()

In [ ]:
model = load_checkpoint_and_dispatch(
    model, "sharded-gpt-j-6B", device_map="balanced", max_memory={"xla:0": "7GiB", "xla:1": "7GiB","xla:2": "7GiB","xla:3": "7GiB","xla:4": "7GiB","xla:5": "7GiB","xla:6": "7GiB","xla:7": "7GiB"}, offload_state_dict=True, no_split_module_classes=["GPTJBlock"]
)

In [ ]:
input_text = "Hi"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output = model.generate(input_ids.to(model.device), do_sample=True, max_length=10)
generated_text = tokenizer.decode(output[0], skip_special_tokens=False)
print("Generated text:", generated_text)